In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

In [ ]:
project_id_lst = pd.read_csv('chatbot_projects.csv', dtype={'id': str})['id'].unique().tolist()

### Section Reference Mapping Table

In [27]:
cols2=pd.read_pickle('cols2.pkl')
cols=pd.read_pickle('col_reference.pkl')

In [29]:
cols.head()

,source,target,section_reference,category,section,subsection
0,52q_acc_area_heat_type,Accessory Area Heating Type,5.2,Mechanical and Electrical Systems,"Heating, Ventilation, and Air Conditioning (HVAC)",
1,31c,Acreage,3.1,Property Characteristics,Parcel Configuration,
2,31ec_acreage_source,Acreage source,3.1,Property Characteristics,Parcel Configuration,
3,322h_ada,ADA (including van spaces),3.2.2,Property Characteristics,Site Improvements,"Vehicular Access, Paving"
4,1y_summary_ada_parking,ADA-designated parking count - Property Data,1.1,Property Description,GENERAL PROPERTY DATA - EXECUTIVE SUMMARY TABLE,


In [28]:
cols2.head()

,section_reference,category,section,subsection
0,5.2,Mechanical and Electrical Systems,"Heating, Ventilation, and Air Conditioning (HVAC)",
1,3.1,Property Characteristics,Parcel Configuration,
3,3.2.2,Property Characteristics,Site Improvements,"Vehicular Access, Paving"
4,1.1,Property Description,GENERAL PROPERTY DATA - EXECUTIVE SUMMARY TABLE,
5,1.2,Property Description,BUILDING SUMMARY TABLE,


### Processing Cost Tables

In [ ]:
ts = pd.read_csv('csv/transmittal-sheets.csv',
           dtype= {'a_column':str, 'id':str, 'a_column':str, 'project_id':str},
           parse_dates=['created_at','updated_at']
           )

In [2]:
# ts = pd.read_pickle('pkl/ts.pkl')

In [9]:
ts = ts[ts['project_id'].isin(project_id_lst)]

In [10]:
ts_st = ts[ts['name'] == 'Short-term'].copy()
ts_lt = ts[ts['name'] == 'Long-term'].copy()

In [12]:
st_cols = ['project_id','a_column', 'b_column', 'c_column', 'd_column', 'e_column', 'g_column', 'h_column']
ts_st = ts_st[st_cols]
# ts_st['total_cost'] = np.where(ts_st['h_column'].isnull() | (ts_st['h_column'].isnull()==0), ts_st['g_column'], ts_st['h_column'])

conditions = [
    (ts_st['c_column'] == 'Immediate (0-12 months)'),
    (ts_st['c_column'] == 'Short-term (12-24 months)')
]
choices = [
    ts_st['g_column'],
    ts_st['h_column']
]
ts_st['total_cost'] = pd.to_numeric(np.select(conditions, choices, default=0)).astype(int)

ts_st = ts_st.drop(['g_column', 'h_column'], axis=1)
ts_st['number of units'] = ts_st['d_column'].astype(str)+' '+ts_st['e_column']
ts_st = ts_st.drop(['d_column', 'e_column'], axis=1)
ts_st.rename(columns={'a_column':'section_reference',
                      'b_column':'cost_item_description',
                      'c_column':'cost_type',
                      }, inplace=True)

ts_st['docu_txt'] = 'There is an ' + ts_st['cost_type'] + ' cost item regarding ' + ts_st['cost_item_description'] + \
                        ' for total unit of ' + ts_st['number of units'].astype(str) + ' with total cost of ' + ts_st['total_cost'].astype(str) + ' dollars.'
ts_st_gb = ts_st.groupby(['project_id', 'section_reference'])['docu_txt'].apply(lambda x: ', '.join(x)).reset_index()

In [14]:
ts_st_gb.head()

,project_id,section_reference,docu_txt
0,246024,3.2.2,There is an Short-term (12-24 months) cost ite...
1,246024,4.3.1,There is an Short-term (12-24 months) cost ite...
2,246036,3.2.2,There is an Immediate (0-12 months) cost item ...
3,246036,3.2.3,There is an Immediate (0-12 months) cost item ...
4,246036,4.3.1,There is an Immediate (0-12 months) cost item ...


In [ ]:
lt_cols = ['project_id','a_column', 'b_column', 'e_column', 'f_column',  'g_column',  'n_column','j_column', 'tab_name',
           'year_1', 'year_2', 'year_3', 'year_4', 'year_5', 'year_6', 'year_7', 'year_8', 'year_9', 'year_10',
           'year_11', 'year_12']
ts_lt = ts_lt[lt_cols]
ts_lt['number of units'] = ts_lt['f_column'].astype(str)+' '+ts_lt['g_column']
ts_lt = ts_lt.drop(['f_column', 'g_column'], axis=1)
ts_lt['e_column'] = ts_lt['e_column'].astype(int)
ts_lt.rename(columns={'a_column':'section_reference',
                      'b_column':'cost_item_description',
                      'e_column':'remaining useful life (years)',
                      'j_column':'item cost type',
                      'n_column':'total cost',
                      'tab_name':'cost type',
                      }, inplace=True)

ts_lt['docu_txt'] = 'There is an ' + ts_lt['item cost type'] + ' Reserve cost item regarding ' + ts_lt['cost_item_description'] + \
                        ' for total unit of ' + ts_lt['number of units'] + ' that have ' + ts_lt['remaining useful life (years)'].astype(str) \
                    + ' years remaining useful life with total cost of ' + ts_lt['total cost'].astype(str) + ' dollars, year 1 costs $' + \
                    ts_lt['year_1'].astype(str) + ', year 2 costs $' + ts_lt['year_2'].astype(str) + ', year 3 costs $' + ts_lt['year_3'].astype(str) \
                    + ', year 4 costs $' + ts_lt['year_4'].astype(str) + ', year 5 costs $' + ts_lt['year_5'].astype(str) + ', year 6 costs $' + \
                    ts_lt['year_6'].astype(str) + ', year 7 costs $' + ts_lt['year_7'].astype(str) + ', year 8 costs $' + ts_lt['year_8'].astype(str) + \
                    ', year 9 costs $' + ts_lt['year_9'].astype(str) + ', year 10 costs $' + ts_lt['year_10'].astype(str) + ', year 11 costs $' + \
                    ts_lt['year_11'].astype(str) + ', year 12 costs $' + ts_lt['year_12'].astype(str) + '.'
ts_lt['docu_txt'] = ts_lt['docu_txt'].replace(", year \d+ costs \$0\.0", "", regex=True)
ts_lt = ts_lt[ts_lt['docu_txt'].notnull()]
ts_lt_gb = ts_lt.groupby(['project_id', 'section_reference'])['docu_txt'].apply(lambda x: ', '.join(x)).reset_index()

In [31]:
ts_lt_union = pd.concat([ts_st_gb, ts_lt_gb], ignore_index=True)
ts_lt_union['cost_related'] = True

In [32]:
ts_lt_union.head()

,project_id,section_reference,docu_txt,cost_related
0,246024,3.2.2,There is an Short-term (12-24 months) cost ite...,True
1,246024,4.3.1,There is an Short-term (12-24 months) cost ite...,True
2,246036,3.2.2,There is an Immediate (0-12 months) cost item ...,True
3,246036,3.2.3,There is an Immediate (0-12 months) cost item ...,True
4,246036,4.3.1,There is an Immediate (0-12 months) cost item ...,True


#### Processing PCA_Data_Items

In [ ]:
pca_items = pd.read_csv('csv/pca-data-items.csv',
             dtype= {'id':str, 'project_id':str, 'document_spot':str,
                    'section_reference':str, })

In [33]:
pca_items = pd.read_pickle('pkl/pca_items.pkl')

MemoryError: 

In [ ]:
pca_items = pca_items[pca_items['document_spot'].isin(cols['source'].tolist()) &
                      pca_items['project_id'].isin(project_id_lst)]

In [ ]:
pca_items['document_data'] = np.where(pca_items['document_text'].isnull(), pca_items['the_data'], pca_items['document_text'])
pca_items = pca_items.drop(['document_text', 'the_data', 'id', 'question_number', 'section_order'], axis=1)

pca_items = pca_items.dropna(subset = ['document_data'])
pca_items = pca_items[pca_items['section_reference'].isin(cols['section_reference'].tolist())
                      | pca_items['section_reference'].str.startswith('1')
                        | pca_items['section_reference'].str.startswith('2.3')]
pca_items = pca_items[~pca_items['document_spot'].str.startswith('pca_reserve')]

del pca_items['section_reference']
pca_items.rename(columns={'document_spot': 'source'}, inplace=True)

pca_items = pca_items.merge(cols, on='source', how='left')

del pca_items['source']

pca_items['docu_txt'] = pca_items['target'] + ' is ' + pca_items['document_data']

pca_items_gb = pca_items.groupby(['project_id', 'section_reference'])['docu_txt'].apply(lambda x: ', '.join(x)).reset_index()

pca_items_gb['cost_related'] = False

### Join parts

In [ ]:
df_union = pd.concat([pca_items_gb, ts_st_gb, ts_lt_gb ], ignore_index=True)

In [ ]:
df_union = df_union.merge(cols2, on = 'section_reference', how = 'left')
df_union = df_union[['project_id', 'section_reference', 'category', 'section', 'subsection',
                      'cost_related','docu_txt']]

#### Prep CSV(document) export for each project

In [ ]:
parent_dir = Path('chatbot_doc_export')
for project_id in project_id_lst:
    export_df = df_union[df_union['project_id'] == project_id].iloc[:, 1:]
    export_df.to_csv(f'{parent_dir}/{project_id}data.csv', index = False)